# <center>Лабораторная работа №2<center>

Выполнила студентка группы 8О-407Б \
Бондарь Ксения

## Задание

Необходимо реализовать алгоритмы машинного обучения:
* \+ ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ
* \+ KNN
* \- SVM
* \+ ДЕРЕВО РЕШЕНИЙ
* \+ RANDOM FOREST

Применить данные алгоритмы на наборы данных, подготовленных в первой лабораторной работе. \
Провести анализ полученных моделей, вычислить метрики классификатора. \
Произвести тюнинг параметров в случае необходимости. \
Сравнить полученные результаты с моделями, реализованными в scikit-learn. \
Аналогично построить метрики классификации. \
Показать, что полученные модели не переобучились. \
Также необходимо сделать выводы о применимости данных моделей к вашей задаче.

## Подготовка данных

Тестовая и тренировочная выборки были подготовлены в Лабораторной работе №1 
(первый датасет).

Импортируем необходимые библиотеки и модули:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.optimize import fmin_tnc
from sklearn.metrics import classification_report
from operator import itemgetter
from scipy.stats import mode

Импортируем данные:

In [2]:
data_train = pd.read_csv("data/data_test1.csv")
data_test = pd.read_csv("data/data_train1.csv")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
data_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
data_test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [5]:
data_train.head()

,Gender,Age,EstimatedSalary,Purchased
0,1,52,138000,1
1,1,53,82000,1
2,0,41,52000,0
3,1,48,30000,1
4,1,48,131000,1


Перед обучением выделим целевую переменную и нормализуем данные.

In [6]:
y_train = data_train['Purchased'].to_numpy()
y_test = data_test['Purchased'].to_numpy()

x_train = data_train.drop('Purchased', axis=1)
x_test = data_test.drop('Purchased', axis=1)

In [7]:
scale_features_std = MinMaxScaler()

x_train = scale_features_std.fit_transform(x_train.to_numpy()) 
x_test = scale_features_std.transform(x_test.to_numpy())

## Модели

### Логистическая регрессия

Логистическая регрессия - это метод обучения, который получается при использовании логистической функции потерь. Основным его свойством является тот факт, что он корректно оценивает вероятность принадлежности объекта к каждому из классов.

#### Реализация

In [8]:
class LogReg:
    @staticmethod
    def logistic_func(x):
        return 1 / (1 + np.exp(-x))
    @staticmethod
    def net_input(B, x):
        return np.dot(x, B)
    def probability(self, B, x):
        return self.logistic_func(self.net_input(B, x))
    def cost_function(self, B, x, y):
        m = x.shape[0]
        total_cost = -(1 / m) * np.sum(
            y * np.log(self.probability(B, x)) + (1 - y) * np.log(
                1 - self.probability(B, x)))
        return total_cost
    def gradient(self, B, x, y):
        m = x.shape[0]
        return (1 / m) * np.dot(x.T, self.logistic_func(self.net_input(B, x)) - y)
    def fit(self, x, y, B):
        weights = fmin_tnc(func=self.cost_function, x0=B, fprime=self.gradient,
                               args=(x, y.flatten()))
        self.w_ = weights[0]
        return self
    def predict(self, x, probab_threshold=0.5):
        theta = self.w_[:, np.newaxis]
        predicted_classes = self.probability(theta, x)
        predicted_classes = (predicted_classes >= probab_threshold).astype(int)
        predicted_classes = predicted_classes.flatten()
        return predicted_classes

In [9]:
B = np.zeros(x_train.shape[1])

lr = LogReg()
lr.fit(x_train, y_train, B)

predicted_classes = lr.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = lr.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("Log reg realization:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

Log reg realization:

Train:

accuracy: 0.75
report:
              precision    recall  f1-score   support

           0       0.52      0.71      0.60        21
           1       0.88      0.76      0.82        59

    accuracy                           0.75        80
   macro avg       0.70      0.74      0.71        80
weighted avg       0.79      0.75      0.76        80

*******************************************************

Test:

accuracy: 0.7625
report:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       222
           1       0.62      0.58      0.60        98

    accuracy                           0.76       320
   macro avg       0.72      0.71      0.72       320
weighted avg       0.76      0.76      0.76       320



#### Реализация из scikit-learn

In [10]:
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
predicted_classes = log_reg.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = log_reg.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("Scikit-learn:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)


Scikit-learn:

Train:

accuracy: 0.825
report:
              precision    recall  f1-score   support

           0       0.66      0.83      0.73        23
           1       0.92      0.82      0.87        57

    accuracy                           0.82        80
   macro avg       0.79      0.83      0.80        80
weighted avg       0.84      0.82      0.83        80

*******************************************************

Test:

accuracy: 0.825
report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.88       230
           1       0.68      0.70      0.69        90

    accuracy                           0.82       320
   macro avg       0.78      0.79      0.79       320
weighted avg       0.83      0.82      0.83       320



### KNN

Метод k-ближайших соседей используется для решения задачи классификации. Он относит объекты к классу, которому принадлежит большинство из k его ближайших соседей в многомерном пространстве признаков.

#### Реализация

In [12]:
class Point:
    def __init__(self, axis):
        self.axis = np.array(axis)   
    def distance(self, other):
        if not isinstance(other, Point):
            other = Point(other)
        return sum((self - other) ** 2) ** 0.5
    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.axis - other.axis)
        return Point(self.axis - np.array(other))
    def __pow__(self, power, modulo=None):
        if modulo:
            return self.axis ** power % modulo
        return self.axis ** power

class KNN:
    def __init__(self, k=3):
        self.k = int(k)
        self._fit_data = []
    def fit(self, x, y):
        assert len(x) == len(y)
        self._fit_data = [(Point(coordinates), label) for coordinates, label in zip(x, y)]
    def predict(self, x):
        predicts = []
        for coordinates in x:
            predict_point = Point(coordinates)
            distances = []
            for data_point, data_label in self._fit_data:
                distances.append((predict_point.distance(data_point), data_label))
            distances = sorted(distances, key=itemgetter(0))[:self.k]
            predicts.append([max(distances, key=itemgetter(1))[1]])
        return predicts

In [13]:
knn = KNN(k=5)
knn.fit(x_train, y_train)

predicted_classes = knn.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = knn.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("knn realization:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

knn realization:

Train:

accuracy: 0.8875
report:
              precision    recall  f1-score   support

           0       0.69      1.00      0.82        20
           1       1.00      0.85      0.92        60

    accuracy                           0.89        80
   macro avg       0.84      0.93      0.87        80
weighted avg       0.92      0.89      0.89        80

*******************************************************

Test:

accuracy: 0.778125
report:
              precision    recall  f1-score   support

           0       0.71      0.98      0.82       165
           1       0.96      0.57      0.71       155

    accuracy                           0.78       320
   macro avg       0.83      0.77      0.77       320
weighted avg       0.83      0.78      0.77       320



#### Реализация из scikit-learn

In [14]:
knn_sklearn = KNeighborsClassifier(n_neighbors=5)
knn_sklearn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [15]:
predicted_classes = knn_sklearn.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = knn_sklearn.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("Scikit-learn:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

Scikit-learn:

Train:

accuracy: 0.9625
report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        32
           1       0.94      1.00      0.97        48

    accuracy                           0.96        80
   macro avg       0.97      0.95      0.96        80
weighted avg       0.96      0.96      0.96        80

*******************************************************

Test:

accuracy: 0.903125
report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       225
           1       0.85      0.82      0.83        95

    accuracy                           0.90       320
   macro avg       0.89      0.88      0.88       320
weighted avg       0.90      0.90      0.90       320



### Дерево решений

Дерево решений — алгоритм классификации, задающийся деревом (связным ациклическим графом).

#### Реализация

In [16]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.feature_prun = 0.1

    def fit(self, X, y, random_feature = False):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y, random_feature)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    
    def _best_split(self, X, y,random_feature):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            if(np.random.randint(0, 11) <= self.feature_prun*10):
                continue
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y,random_feature, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y,random_feature)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left,random_feature, depth + 1)
                node.right = self._grow_tree(X_right, y_right,random_feature, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [17]:
dt = DecisionTree(max_depth=5)
dt.fit(x_train, y_train)

predicted_classes = dt.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = dt.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("ds realization:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

ds realization:

Train:

accuracy: 1.0
report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        51

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

*******************************************************

Test:

accuracy: 0.825
report:
              precision    recall  f1-score   support

           0       0.79      0.96      0.86       186
           1       0.92      0.63      0.75       134

    accuracy                           0.82       320
   macro avg       0.85      0.80      0.81       320
weighted avg       0.84      0.82      0.82       320



#### Реализация из scikit-learn

In [18]:
ds_sklearn = tree.DecisionTreeClassifier(max_depth=5)
ds_sklearn = ds_sklearn.fit(x_train, y_train)

In [19]:
predicted_classes = ds_sklearn.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = ds_sklearn.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("Scikit-learn:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

Scikit-learn:

Train:

accuracy: 1.0
report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        51

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

*******************************************************

Test:

accuracy: 0.821875
report:
              precision    recall  f1-score   support

           0       0.78      0.96      0.86       185
           1       0.92      0.63      0.75       135

    accuracy                           0.82       320
   macro avg       0.85      0.80      0.81       320
weighted avg       0.84      0.82      0.81       320



### Random forest

RF (random forest) — это множество решающих деревьев. В задаче регрессии их ответы усредняются, в задаче классификации принимается решение голосованием по большинству.

#### Реализация

In [20]:
class RandForest():

    def __init__(
        self, n_estimators=32, max_features=8, max_depth=10,
        min_samples_split=2, bootstrap=0.9
        ):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.bootstrap = bootstrap
        self.forest = []


    def fit(self, X, y):
        self.forest = []
        n_samples = len(y)
        n_sub_samples = round(n_samples*self.bootstrap)
        
        for i in range(self.n_estimators):
            X_subset = X[:n_sub_samples]
            y_subset = y[:n_sub_samples]

            tree_ = tree.DecisionTreeClassifier()
            tree_.fit(X_subset, y_subset)
            self.forest.append(tree_)


    def predict(self, X):
        n_samples = X.shape[0]
        n_trees = len(self.forest)
        predictions = np.empty([n_trees, n_samples])
        for i in range(n_trees):
            predictions[i] = self.forest[i].predict(X)

        return mode(predictions)[0][0]


    def score(self, X, y):
        y_predict = self.predict(X)
        n_samples = len(y)
        correct = 0
        for i in xrange(n_samples):
            if y_predict[i] == y[i]:
                correct = correct + 1
        accuracy = correct/n_samples
        return accuracy

In [21]:
rf = RandForest()
rf.fit(x_train, y_train)

predicted_classes = rf.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = rf.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("rf realization:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

rf realization:

Train:

accuracy: 1.0
report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        29
         1.0       1.00      1.00      1.00        51

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80

*******************************************************

Test:

accuracy: 0.825
report:
              precision    recall  f1-score   support

         0.0       0.79      0.96      0.86       186
         1.0       0.92      0.63      0.75       134

    accuracy                           0.82       320
   macro avg       0.85      0.80      0.81       320
weighted avg       0.84      0.82      0.82       320



#### Реализация из scikit-learn:

In [22]:
rf_sklearn = RandomForestClassifier(n_estimators=50, max_depth=2)
rf_sklearn.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
predicted_classes = rf_sklearn.predict(x_train)
train_accuracy = accuracy_score(predicted_classes, y_train.flatten())
train_report = classification_report(predicted_classes, y_train.flatten())

predicted_classes = rf_sklearn.predict(x_test)
test_accuracy = accuracy_score(predicted_classes, y_test.flatten())
test_report = classification_report(predicted_classes, y_test.flatten())

print("Scikit-learn:")
print("\nTrain:\n\naccuracy: {}".format(train_accuracy))
print("report:")
print(train_report)
print('*'*55)
print("\nTest:\n\naccuracy: {}".format(test_accuracy))
print("report:")
print(test_report)

Scikit-learn:

Train:

accuracy: 0.975
report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        29
           1       0.98      0.98      0.98        51

    accuracy                           0.97        80
   macro avg       0.97      0.97      0.97        80
weighted avg       0.97      0.97      0.97        80

*******************************************************

Test:

accuracy: 0.7125
report:
              precision    recall  f1-score   support

           0       0.62      0.96      0.76       148
           1       0.93      0.50      0.65       172

    accuracy                           0.71       320
   macro avg       0.78      0.73      0.70       320
weighted avg       0.79      0.71      0.70       320



### Вывод

В данной лабораторной работе было реализовано несколько базовых алгоритмов машинного обучения и построены модели из библиотеки Scikit-learn. Почти во всех случаях по метрикам иобучающей и тестовой выборок можно предположить, что модели переобучились. 